In [1]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.2 MB/s eta 0:00:00


In [2]:
import time, math
from typing import Any

import numpy as np
import faiss
from sklearn.datasets import make_blobs

## Config

In [3]:
NB = 50_000
NQ = 20000
D = 128
K = 10
SEED = 123

# HNSW
HNSW_M = 16
HNSW_EF_CONS = 200
HNSW_EF_SRCH = 64

# IVF
IVF_NLIST = max(1, int(NB ** 0.5))
IVF_NPROBE = 1

## Helpers

In [5]:
def now() -> float:
    return time.perf_counter()

def human_bytes(n: int) -> str:
    if n is None: return "-"
    if n == 0: return "0 B"
    units = ["B","KB","MB","GB","TB"]
    k = 1024.0
    i = int(math.floor(math.log(n, k)))
    i = min(i, len(units)-1)
    return f"{n / (k**i):.2f} {units[i]}"

def recall_at_k(I_pred: np.ndarray, I_true: np.ndarray) -> float:
    nq, k = I_true.shape
    hit = 0
    for i in range(nq):
        hit += len(set(I_pred[i]).intersection(I_true[i]))
    return hit / (nq * k)

def serialize_size_bytes(index: faiss.Index) -> int | None:
    try:
        b = faiss.serialize_index(index)
        return len(b)
    except Exception:
        return None

def compute_ground_truth(xb: np.ndarray, xq: np.ndarray, k: int) -> tuple[np.ndarray, float]:
    ref = faiss.IndexFlatL2(xb.shape[1])
    ref.add(xb)
    t0 = now()
    _, I_true = ref.search(xq, k)
    return I_true, (now() - t0)

def print_table(rows: list[dict[str, Any]]) -> None:
    headers = [
        "method","params","train_s","add_s","build_s",
        "search_s","avg_ms_per_q","recall@k","index_size"
    ]
    colw = {h: len(h) for h in headers}
    fmt_rows = []
    for r in rows:
        rf = {
            "method": r["method"],
            "params": r.get("params","") or "-",
            "train_s": f'{r["train_s"]:.3f}',
            "add_s": f'{r["add_s"]:.3f}',
            "build_s": f'{r["build_s"]:.3f}',
            "search_s": f'{r["search_s"]:.3f}',
            "avg_ms_per_q": f'{r["avg_ms_per_q"]:.3f}',
            "recall@k": f'{r["recall_at_k"]:.4f}',
            "index_size": human_bytes(r["index_size_bytes"]) if r["index_size_bytes"] is not None else "-"
        }
        for h in headers:
            colw[h] = max(colw[h], len(str(rf[h])))
        fmt_rows.append(rf)
    line = " | ".join(h.ljust(colw[h]) for h in headers)
    sep  = "-+-".join("-"*colw[h] for h in headers)
    print(line); print(sep)
    for rf in fmt_rows:
        print(" | ".join(str(rf[h]).ljust(colw[h]) for h in headers))

In [6]:
def _sample_centers(num_clusters: int, dim: int, center_box=(-20.0, 20.0), seed: int = 123) -> np.ndarray:
    rng = np.random.default_rng(seed)
    low, high = center_box
    centers = rng.uniform(low, high, size=(num_clusters, dim)).astype(np.float32)
    return centers

def generate_data(
    num_base_vectors: int,
    num_query_vectors: int,
    dim: int,
    num_clusters: int,
    cluster_std_base: float = 0.50,
    cluster_std_query: float = 0.15,
    center_box = (-20.0, 20.0),
    seed: int = 123,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Tworzy mieszankę gaussów z tymi samymi centrami dla bazy i zapytań.
    To sprzyja IVF (nprobe=1-2) przy nlist ~= liczba klastrów.
    """
    centers = _sample_centers(num_clusters, dim, center_box=center_box, seed=seed)
    xb, _ = make_blobs(
        n_samples=num_base_vectors,
        centers=centers,
        cluster_std=cluster_std_base,
        random_state=seed,
        n_features=dim,
    )
    xq, _ = make_blobs(
        n_samples=num_query_vectors,
        centers=centers,
        cluster_std=cluster_std_query,
        random_state=seed + 1,
        n_features=dim,
    )
    return xb.astype(np.float32), xq.astype(np.float32)

## Building the index

In [7]:
def build_index_knn(d: int) -> faiss.Index:
    return faiss.IndexFlatL2(d)

def build_index_hnsw(d: int, M: int = HNSW_M, efC: int = HNSW_EF_CONS, efS: int = HNSW_EF_SRCH) -> faiss.Index:
    idx = faiss.IndexHNSWFlat(d, M)
    idx.hnsw.efConstruction = efC
    idx.hnsw.efSearch = efS
    return idx

def build_index_ivf(d: int, nlist: int = IVF_NLIST, metric=faiss.METRIC_L2) -> faiss.Index:
    quantizer = faiss.IndexFlatL2(d)
    idx = faiss.IndexIVFFlat(quantizer, d, nlist, metric)
    return idx

## Index operations

In [8]:
def train_if_needed(index: faiss.Index, xb: np.ndarray) -> float:
    """Trenuje index jeśli wymaga treningu (np. IVF). Zwraca czas treningu."""
    if index.is_trained:
        return 0.0
    t0 = now()
    index.train(xb)
    return now() - t0

def add_vectors(index: faiss.Index, xb: np.ndarray) -> float:
    t0 = now()
    index.add(xb)
    return now() - t0

def search_index(index: faiss.Index, xq: np.ndarray, k: int) -> tuple[np.ndarray, np.ndarray, float]:
    # krótki warmup niweluje jednorazowe koszty
    _ = index.search(xq[: min(10, len(xq))], k)
    t0 = now()
    D, I = index.search(xq, k)
    t = now() - t0
    return D, I, t

## Benchmarking

In [9]:
def benchmark_index(
    index: faiss.Index,
    name: str,
    xb: np.ndarray,
    xq: np.ndarray,
    k: int,
    extra_setup=None
) -> dict[str, Any]:
    """
    Mierzy: train_s, add_s, build_s, search_s, qps, avg_ms_per_q, index_size_bytes.
    extra_setup: opcjonalna funkcja(index) -> None (np. ustawienie nprobe).
    """
    if extra_setup:
        extra_setup(index)

    t_train = train_if_needed(index, xb)
    t_add = add_vectors(index, xb)
    _, I_pred, t_search = search_index(index, xq, k)

    return {
        "method": name,
        "train_s": t_train,
        "add_s": t_add,
        "build_s": t_train + t_add,
        "search_s": t_search,
        "qps": (len(xq) / t_search) if t_search > 0 else float("inf"),
        "avg_ms_per_q": (t_search / len(xq) * 1000.0),
        "I_pred": I_pred,  # do wyliczenia recall
        "index_size_bytes": serialize_size_bytes(index),
        "params": "",
    }

In [10]:
def run_benchmark_default(nb=NB, nq=NQ, d=D, k=K, seed=SEED) -> list[dict[str, Any]]:
    xb, xq = generate_data(
        num_base_vectors=NB,
        num_query_vectors=NQ,
        dim=D,
        num_clusters=IVF_NLIST*2,
        cluster_std_base=0.50,
        cluster_std_query=0.15,
        center_box=(-20.0, 20.0),
        seed=SEED,
    )

    I_true, t_gt = compute_ground_truth(xb, xq, k)
    print(f"[info] Ground truth (IndexFlatL2) search time: {t_gt:.3f}s")

    results: List[Dict[str, Any]] = []

    idx_knn = build_index_knn(d)
    r_knn = benchmark_index(idx_knn, "KNN(IndexFlatL2)", xb, xq, k)
    r_knn["params"] = "-"
    r_knn["recall_at_k"] = recall_at_k(r_knn["I_pred"], I_true)
    results.append(r_knn)

    idx_hnsw = build_index_hnsw(d, M=HNSW_M, efC=HNSW_EF_CONS, efS=HNSW_EF_SRCH)
    r_hnsw = benchmark_index(idx_hnsw, "HNSW(IndexHNSWFlat)", xb, xq, k)
    r_hnsw["params"] = f"M={HNSW_M}, efC={HNSW_EF_CONS}, efS={HNSW_EF_SRCH}"
    r_hnsw["recall_at_k"] = recall_at_k(r_hnsw["I_pred"], I_true)
    results.append(r_hnsw)

    if nb < IVF_NLIST:
        print(f"[warn] nb({nb}) < nlist({IVF_NLIST}); zmniejszam nlist do nb")
        nlist = nb
    else:
        nlist = IVF_NLIST

    def _ivf_setup(idx):
        if hasattr(idx, "nprobe"):
            idx.nprobe = IVF_NPROBE

    idx_ivf = build_index_ivf(d, nlist=nlist)
    r_ivf = benchmark_index(idx_ivf, "IVF(IndexIVFFlat)", xb, xq, k, extra_setup=_ivf_setup)
    r_ivf["params"] = f"nlist={nlist}, nprobe={IVF_NPROBE}"
    r_ivf["recall_at_k"] = recall_at_k(r_ivf["I_pred"], I_true)
    results.append(r_ivf)

    for r in results:
        r.pop("I_pred", None)

    return results


In [11]:
rows = run_benchmark_default()
print()
print_table(rows)

[info] Ground truth (IndexFlatL2) search time: 7.210s

method              | params                | train_s | add_s  | build_s | search_s | avg_ms_per_q | recall@k | index_size
--------------------+-----------------------+---------+--------+---------+----------+--------------+----------+-----------
KNN(IndexFlatL2)    | -                     | 0.000   | 0.013  | 0.013   | 8.009    | 0.400        | 1.0000   | 24.41 MB  
HNSW(IndexHNSWFlat) | M=16, efC=200, efS=64 | 0.000   | 11.349 | 11.349  | 0.917    | 0.046        | 0.9962   | 31.30 MB  
IVF(IndexIVFFlat)   | nlist=223, nprobe=1   | 0.813   | 0.104  | 0.917   | 0.338    | 0.017        | 0.9909   | 24.91 MB  
